# [2] Image Classification

- 본 섹션은 로컬에서 수집된 데이터를 통해 새로운 이미지 분류 모델을 훈련시키고 예측을 수행합니다.
- 관련 자료 : [Tensorflow Image Classification Docs](https://www.tensorflow.org/tutorials/images/classification)

## 2.1. 훈련 (Train)

### 2.1.1. 훈련 정의

In [ ]:
images_data_dir = 'saved_data/images/2025-02-16T18-59-31'

epochs = 30
img_resize_height = 120
img_resize_width = 160
batch_size = 16
validation_split = 0.2
split_random_seed = 21

### 2.1.2. 모델 저장 폴더 생성 및 메타데이터 복사

In [ ]:
import os, shutil, json

model_save_dir = os.path.join('saved_data/classification_models/', os.path.basename(images_data_dir))
os.makedirs(model_save_dir, exist_ok=True)
print("model_save_dir :", model_save_dir)

# metadata.json
shutil.copy(os.path.join(images_data_dir, 'metadata.json'), model_save_dir)
with open(os.path.join(model_save_dir, 'metadata.json'), 'r') as f:
    metadata = json.load(f)
    print('metadata.json copied')

action_space = {action["index"]: action for action in metadata['action_space']}
class_names = [str(action_index) for action_index in sorted(action_space.keys())]
print('action_space:', action_space)
print('class_names:', class_names)

### 2.1.3. 데이터셋 준비 (train / validation 분할)

In [ ]:
import tensorflow as tf

# 훈련 데이터셋
train_ds = tf.keras.utils.image_dataset_from_directory(
  images_data_dir,
  validation_split=validation_split,
  subset="training",
  seed=split_random_seed,
  image_size=(img_resize_height, img_resize_width),
  batch_size=batch_size,
  class_names=class_names
)

# 검증 데이터셋
val_ds = tf.keras.utils.image_dataset_from_directory(
  images_data_dir,
  validation_split=validation_split,
  subset="validation",
  seed=split_random_seed,
  image_size=(img_resize_height, img_resize_width),
  batch_size=batch_size,
  class_names=class_names
)

# Prefectch & Cache (성능 최적화)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


### 2.1.4. 모델 구성

사전 훈련된 모델을 가져와서 로컬 데이터로 훈련 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

num_classes = len(class_names) # 클래스 개수 자동 추출

# MobileNetV2 Feature Extractor
base_model = keras.applications.MobileNetV2(
    weights='imagenet', 
    input_shape=(img_resize_height, img_resize_width, 3),  
    include_top=False
)

# base_model 가중치를 고정 여부
base_model.trainable = True

# 모델 구성
inputs = keras.Input(shape=(None, None, 3))  # 유동적인 입력 크기 허용

# Resize 레이어 적용
x = layers.Resizing(img_resize_height, img_resize_width, name='resizing')(inputs)

# 데이터 정규화 (0~1 범위)
x = layers.Rescaling(1./255)(inputs)

# MobileNetV2 백본 사용
x = base_model(x, training=False)

# 특징 맵을 벡터로 변환
x = layers.GlobalAveragePooling2D()(x)

# 완전 연결층
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)  # 과적합 방지

# 최종 출력층 (클래스 개수에 맞춤)
outputs = layers.Dense(num_classes, activation='softmax')(x)

# 모델 생성
model = keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # 정수형 라벨 사용 시
    metrics=['accuracy']
)

model.summary()


### 2.1.5. 콜백 설정

In [ ]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = []

# 1) 조기 종료
early_stopping = EarlyStopping(
    monitor='val_loss',         # 모니터링 대상
    patience=5,                 # 에포크 동안 성능 향상이 없으면 조기 종료
    restore_best_weights=True,  # 중단 시점에 최적 가중치로 복원
    verbose=1                   # 로그 출력
)
callbacks.append(early_stopping)

# 2) 최고 성능(best) 모델 저장
ckpt_best = ModelCheckpoint(
    filepath=os.path.join(model_save_dir, 'best.keras'),
    monitor='val_loss',
    save_best_only=True,       # 최적 성능 갱신 시에만 저장
    save_weights_only=False,   # 가중치만 저장할지 여부.
    save_freq='epoch',         # 저장 빈도 (epoch 단위)
    verbose=1,                 # 로그 출력
    mode='min',                # val_loss이므로 'min' 모드가 적합
)
callbacks.append(ckpt_best)

# 3) 매 에포크마다(실시간) 체크포인트 저장
# ckpt_every_epoch = ModelCheckpoint(
#     filepath=os.path.join(model_save_dir, 'epoch_{epoch:02d}.keras'),   # epoch 번호를 파일명에 포함
#     monitor='val_loss',      # 모니터링 대상
#     save_best_only=False,    # 모든 epoch 저장
#     save_weights_only=False, # 가중치만 저장할지 여부.
#     save_freq='epoch',       # 저장 빈도 (epoch 단위)
#     verbose=1,               # 로그 출력
#     mode='auto',             # 보통은 auto / min / max 중 선택
# )
# callbacks.append(ckpt_every_epoch)

### 2.1.6. 모델 훈련

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

### 2.1.7. 훈련 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

# 1) Loss 시각화
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# 2) Accuracy 시각화
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


## 2.2. 추론 (Predict)

### 2.2.1. 모델 불러오기

### 2.2.2. Vehicle Client 가져오기

In [ ]:
import os
from dotenv import load_dotenv
from aicastle.deepracer.vehicle.api.client import VehicleClient
load_dotenv(".env")

vehicle =VehicleClient(
    ip=os.getenv("VEHICLE_IP"),
    password=os.getenv("VEHICLE_PASSWORD"),
)


### 2.2.3. 예측 (추론) 함수 정의

In [ ]:
import numpy as np
import time

def predict(model, vehicle, action_space):
    s_time = time.time()

    img = vehicle.get_image(color="rgb")  # shape: (H, W, 3)
    x = np.expand_dims(img, axis=0)      # shape: (1, H, W, 3)
    y = model.predict(x, verbose=0)      # shape: (1, num_classes)
    pred_probs = y[0]
    pred_class_idx = pred_probs.argmax()
    action = action_space[pred_class_idx]

    f_time = time.time()
    inference_time = f_time - s_time

    return pred_probs, pred_class_idx, action, inference_time

- 예측 테스트 (1회)

In [ ]:
pred_probs, pred_class_idx, action, inference_time = predict(model, vehicle, action_space)

print("예측 확률 :", pred_probs)
print("예측 클래스 인덱스:", pred_class_idx)
print("예측 클래스 :", action_space[pred_class_idx])
print("추론 시간 :", round(inference_time,3), "sec")

### 2.2.4. 예측 및 자율주행

In [ ]:
from IPython.display import clear_output

vehicle.set_speed_percent(100)

try :
	for _ in range(100):
		pred_probs, pred_class_idx, action, inference_time = predict(model, vehicle, action_space)
		vehicle.move(angle=action['steering_angle'], speed=action['speed'])
		clear_output(wait=True)
		print("예측 확률 :", pred_probs)
		print("예측 클래스 인덱스:", pred_class_idx)
		print("예측 클래스 :", action_space[pred_class_idx])
		print("추론 시간 :", round(inference_time,3), "sec")
except :
    pass
vehicle.stop()